In [1]:
sc

In [2]:
sc.setLogLevel("ERROR")

In [3]:
#cp  -rf /usr/lib/hive/conf/hive-site.xml   /usr/lib/spark/conf/

# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [4]:
! hdfs dfs -put LoanStats_web.csv /user/root/

put: `/user/root/LoanStats_web.csv': File exists


In [5]:
! hdfs dfs -ls /user/root/

Found 11 items
-rw-r--r--   1 aekanun  supergroup  416674235 2017-09-06 01:18 /user/root/LoanStats_web.csv
-rw-r--r--   1 trainee1 supergroup     977501 2017-11-16 10:34 /user/root/TelcoChurn.csv
-rw-r--r--   1 aekanun  supergroup      56329 2017-10-19 02:31 /user/root/churn-bigml-20.csv
-rw-r--r--   1 aekanun  supergroup     223998 2017-10-19 02:31 /user/root/churn-bigml-80.csv
-rw-r--r--   1 trainee7 supergroup   47251540 2017-10-29 06:04 /user/root/corrected
drwxr-xr-x   - aekanun  supergroup          0 2017-09-06 01:20 /user/root/datacruncher
drwxr-xr-x   - aekanun  supergroup          0 2017-11-14 04:14 /user/root/fight_result
-rw-r--r--   1 trainee3 supergroup       1038 2017-10-14 14:52 /user/root/file-lines.txt
-rw-r--r--   1 trainee1 supergroup  742579829 2017-11-07 17:20 /user/root/kddcup.data
-rw-r--r--   1 trainee1 supergroup       1307 2017-11-07 17:20 /user/root/kddcup.names
-rw-r--r--   1 aekanun  supergroup  441771600 2017-09-06 01:19 /user/root/loan.csv


In [6]:
raw201617_df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true').load('/user/root/LoanStats_web.csv')

In [7]:
#raw201617_df.printSchema()

In [8]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

In [9]:
#rawweb_df.count()

## Collecting the old data from the Kaggle. (IMC's bucket)

In [10]:
! hdfs dfs -rm /user/root/datacruncher/loan/*

rm: `/user/root/datacruncher/loan/*': No such file or directory


In [11]:
! hdfs dfs -ls /user/root/datacruncher/loan

In [12]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder, VectorIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [13]:
rawkaggle_df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true').load('/user/root/loan.csv')

In [14]:
raw_df = rawkaggle_df.unionAll(rawweb_df)

In [15]:
raw_df.columns

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_

In [16]:
raw_df.count()

1418575

In [17]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

## Check for a schema

In [18]:
df_colfam5.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: double (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



## Data Analysis: Check for imbalance data and class's label

In [19]:
from pyspark.sql.functions import count

In [20]:
df_colfam5.groupby(['loan_status']).agg(count('*').alias("cnt")).show()

+--------------------+-------+
|         loan_status|    cnt|
+--------------------+-------+
|Does not meet the...|    761|
|         Charged Off|  67224|
|          Fully Paid| 292045|
|     In Grace Period|  12810|
|Does not meet the...|   1988|
|             Current|1006083|
|   Late (16-30 days)|   5221|
|              Issued|   8460|
|                null|     10|
|             Default|   1228|
|  Late (31-120 days)|  22745|
+--------------------+-------+



In [21]:
df_colfam5.select(['installment']).describe().show()

+-------+------------------+
|summary|       installment|
+-------+------------------+
|  count|           1418565|
|   mean| 439.9462110936163|
| stddev|255.42991573793574|
|    min|             15.67|
|    max|           1715.42|
+-------+------------------+



# 2. Data Preparation

## Data Cleansing: Remove missing values

In [22]:
df_colfam5.count()

1418575

In [23]:
df_no_missing = df_colfam5.dropna(how='any')

In [24]:
df_no_missing.count()

1417966

In [25]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [26]:
df_no_missing_cached = df_no_missing_fitmem.cache()

## Register to temporary table.

In [27]:
df_no_missing_cached.registerTempTable("df")

## Data Transformation: Remove Sign of Percent and Extract Month.

In [28]:
a = 'Jan-1985'
a.split('-')[0]

'Jan'

In [29]:
sqlContext.udf.register("t_revol_util", lambda x: x.rstrip('%'))

In [30]:
sqlContext.udf.register("t_int_rate", lambda x: x.rstrip('%'))

In [31]:
sqlContext.udf.register("t_earliest_cr_line", lambda x: x.split('-')[0])

In [32]:
sqlContext.udf.register("t_last_credit_pull_d", lambda x: x.split('-')[0])

## Filter only "Fully Paid" and "Charged Off" and relevant fields.

In [33]:
dfWithCrunch = sqlContext.sql("""SELECT loan_amnt, term, t_int_rate(int_rate) as int_rate, \
installment, grade, emp_length, home_ownership, annual_inc, verification_status, loan_status, \
purpose, addr_state, dti, delinq_2yrs, t_earliest_cr_line(earliest_cr_line) as earliest_cr_line, \
inq_last_6mths, open_acc, pub_rec, revol_bal,  t_revol_util (revol_util) as revol_util, total_acc, \
t_last_credit_pull_d(last_credit_pull_d) as last_credit_pull_d FROM df \
where loan_status = 'Fully Paid' or loan_status = 'Charged Off' """)

## Number of data rows that are only "Fully Paid" and "Charged Off"

In [34]:
dfWithCrunch.count()

359058

## Writing into a Hive Table

In [35]:
dfWithCrunch.write.format("parquet").mode('Append').parquet('/user/root/datacruncher/loan')

In [36]:
sqlStringADD = 'loan_amnt double,term string, int_rate string, installment double, grade string, emp_length string, home_ownership string, annual_inc double, verification_status string, loan_status string, purpose string, addr_state string, dti double, delinq_2yrs double, earliest_cr_line string, inq_last_6mths double, open_acc double, pub_rec double, revol_bal double, revol_util string, total_acc double, last_credit_pull_d string'

In [37]:
! hive -e "DROP TABLE IF EXISTS personal_loan"
! hive -e "CREATE EXTERNAL TABLE personal_loan (loan_amnt double,term string, int_rate string, installment double, grade string, emp_length string, home_ownership string, annual_inc double, verification_status string, loan_status string, purpose string, addr_state string, dti double, delinq_2yrs double, earliest_cr_line string, inq_last_6mths double, open_acc double, pub_rec double, revol_bal double, revol_util string, total_acc double, last_credit_pull_d string) STORED AS PARQUET LOCATION '/user/root/datacruncher/loan'"

2017-12-06 01:05:07,431 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.

Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 0.699 seconds
WARN: The method class org.apache.commons.logging.impl.SLF4JLogFactory#release() was invoked.
WARN: Please see http://www.slf4j.org/codes.html#release for an explanation.
2017-12-06 01:05:14,860 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.

Logging initialized using configuration in file:/etc/hive/conf.dist/hive-log4j.properties
OK
Time taken: 0.749 seconds
WARN: The method class org.apache.commons.logging.impl.SLF4JLogFactory#release() was invoked.
WARN: Please see http://www.slf4j.org/codes.html#release for an explanation.


## Load the data from Hive and register to "crunched_data" table.

In [38]:
df1 = sqlContext.read.load("/user/root/datacruncher/loan").repartition(60).cache()
df1.registerTempTable("crunched_data")

##### ********* Before this, they are just attributes/Fields for Descriptive Analytics *********

# 3. Modeling

##### ********* After this, they are Features for Predictive Analytics *********

## Feature Extraction (Transformation): Normalization of "annual_inc" and "loan_amnt"

In [39]:
from pyspark.sql.functions import *

max_annual_inc = df1.select(max('annual_inc')).collect()[0][0]

min_annual_inc = df1.select(min('annual_inc')).collect()[0][0]

sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [40]:
max_loan_amnt = df1.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = df1.select(min('loan_amnt')).collect()[0][0]

sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [41]:
data = sqlContext.sql('SELECT double(t_loan_amnt(loan_amnt)) as loan_amnt, term, double(int_rate) as int_rate, installment, grade, emp_length, home_ownership, double(t_annual_inc(annual_inc)) as annual_inc, verification_status, loan_status, purpose, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, open_acc, pub_rec, revol_bal,  double(revol_util) as revol_util, total_acc, last_credit_pull_d FROM crunched_data').repartition(60).cache()

In [42]:
data_no_missing_df = data.dropna(how='any')

## Define Label Column.

In [43]:
labelIndexer = StringIndexer(inputCol="loan_status", outputCol="label")

## Split the data into training set and test set.

In [44]:
training, test = data_no_missing_df.randomSplit([0.70, 0.30], seed = 12)

In [45]:
#training.toPandas().transpose()

## Feature Selection and Feature Extraction (Transformation) - Vectors

In [46]:
termIndexer = StringIndexer(inputCol="term", outputCol="termIndex")
termEncoder = OneHotEncoder(inputCol="termIndex", outputCol="termVec")
gradeIndexer = StringIndexer(inputCol="grade", outputCol="gradeIndex")
gradeEncoder = OneHotEncoder(inputCol="gradeIndex", outputCol="gradeVec")
emp_lengthIndexer = StringIndexer(inputCol="emp_length", outputCol="emp_lengthIndex")
emp_lengthEncoder = OneHotEncoder(inputCol="emp_lengthIndex", outputCol="emp_lengthVec")
home_ownershipIndexer = StringIndexer(inputCol="home_ownership", outputCol="home_ownershipIndex")
home_ownershipEncoder = OneHotEncoder(inputCol="home_ownershipIndex", outputCol="home_ownershipVec")
verification_statusIndexer = StringIndexer(inputCol="verification_status", outputCol="verification_statusIndex")
verification_statusEncoder = OneHotEncoder(inputCol="verification_statusIndex", outputCol="verification_statusVec")
loan_statusIndexer = StringIndexer(inputCol="loan_status", outputCol="loan_statusIndex")
loan_statusEncoder = OneHotEncoder(inputCol="loan_statusIndex", outputCol="loan_statusVec")
purposeIndexer = StringIndexer(inputCol="purpose", outputCol="purposeIndex")
purposeEncoder = OneHotEncoder(inputCol="purposeIndex", outputCol="purposeVec")


addr_stateIndexer = StringIndexer(inputCol="addr_state", outputCol="addr_stateIndex")
addr_stateEncoder = OneHotEncoder(inputCol="addr_stateIndex", outputCol="addr_stateVec")


earliest_cr_lineIndexer = StringIndexer(inputCol="earliest_cr_line", outputCol="earliest_cr_lineIndex")
earliest_cr_lineEncoder = OneHotEncoder(inputCol="earliest_cr_lineIndex", outputCol="earliest_cr_lineVec")
revol_utilIndexer = StringIndexer(inputCol="revol_util", outputCol="revol_utilIndex")
revol_utilEncoder = OneHotEncoder(inputCol="revol_utilIndex", outputCol="revol_utilVec")
last_credit_pull_dIndexer = StringIndexer(inputCol="last_credit_pull_d", outputCol="last_credit_pull_dIndex")
last_credit_pull_dEncoder = OneHotEncoder(inputCol="last_credit_pull_dIndex", outputCol="last_credit_pull_dVec")

In [47]:
assembler = VectorAssembler(inputCols=[\
                                       "loan_amnt",\
                                       "int_rate",\
                                       "gradeVec",\
                                       "termVec",\
                                       "earliest_cr_lineVec",\
                                       "last_credit_pull_dVec",\
                                       "emp_lengthVec",\
                                       "home_ownershipVec",\
                                       "verification_statusVec",\
                                       "annual_inc",\
                                       "purposeVec",\
                                       "addr_stateVec",\
                                       "revol_util",\
                                       "installment",\
                                       "annual_inc",\
                                       "dti","delinq_2yrs",\
                                       "pub_rec"\
                                      ], outputCol="features")

In [48]:
dt = RandomForestClassifier(labelCol="label", featuresCol="features")
 
pipeline = Pipeline().setStages([labelIndexer,\
                                 gradeIndexer,gradeEncoder,\
                                 termIndexer,termEncoder,\
                                 emp_lengthIndexer, emp_lengthEncoder,\
                                 home_ownershipIndexer, home_ownershipEncoder, \
                                 verification_statusIndexer,verification_statusEncoder,\
                                 purposeIndexer,purposeEncoder,\
                                 addr_stateIndexer,addr_stateEncoder,\
                                 earliest_cr_lineIndexer,earliest_cr_lineEncoder,\
                                 last_credit_pull_dIndexer, last_credit_pull_dEncoder,\
                                 assembler, dt])

In [49]:
dt.maxDepth

Param(parent='RandomForestClassifier_404486d227b5806f9268', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.')

In [50]:
#dt.numTrees

## Make balance data

In [83]:
filtered_Ftraining = training.filter(training.loan_status == 'Fully Paid').sample(False, 0.23, 42)

In [84]:
filtered_Ctraining = training.filter(training.loan_status == 'Charged Off')

In [85]:
filtered_Ftraining.count()

46729

In [86]:
filtered_Ctraining.count()

47173

In [87]:
training2 = filtered_Ftraining.unionAll(filtered_Ctraining)

## Make Cross-Validations and Hyperparameter tuning

In [88]:
from time import time
t0 = time()

In [89]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [90]:
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [1,3,5]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=10)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training2)

## Training Completed in Seconds:

In [91]:
tt = time() - t0
#print "Training completed in {} seconds".format(round(tt,3))
tt

1314.8620448112488

In [92]:
result1 = cvModel.bestModel.transform(test)

# 4. Evaluation of Models

In [93]:
test.count()

107813

In [94]:
result1.count()

107813

In [95]:
test.filter(training.loan_status == 'Fully Paid').count()

87826

In [96]:
test.filter(training.loan_status == 'Charged Off').count()

19987

## Calculation for Confusion Matrix.

In [97]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['label']==0) & (lap['prediction']==0)]))

    tn = float(len(lap[(lap['label']==1) & (lap['prediction']==1)]))

    fp = float(len(lap[(lap['label']==1) & (lap['prediction']==0)]))

    fn = float(len(lap[(lap['label']==0) & (lap['prediction']==1)]))

    positivepredictivevalue = tp / (tp+fp)
    negativepredictivevalue = tn / (tn+fn)

    sensitivity = tp / (tp+fn)
    specificity = tn / (tn+fp)

    accuracy = (tp+tn) / (tp+tn+fp+fn)

    return {'PPV': positivepredictivevalue, 'NPV':negativepredictivevalue,'Sensitivity': sensitivity, 'Specificity': specificity, 'Accuracy': accuracy}

In [98]:
lap = result1.select("label", "prediction").toPandas()

In [99]:
m = eval_metrics(lap)

In [100]:
m

{'Accuracy': 0.6229489950191536,
 'NPV': 0.8963952609024451,
 'PPV': 0.2861223813861058,
 'Sensitivity': 0.6915495071796668,
 'Specificity': 0.6073372349873614}

In [101]:
tp = float(len(lap[(lap['label']==0) & (lap['prediction']==0)])) #tp - Full paid

In [102]:
tp

13822.0

In [103]:
tn = float(len(lap[(lap['label']==1) & (lap['prediction']==1)])) #tn - Charged off

In [104]:
tn

53340.0

In [105]:
fp = float(len(lap[(lap['label']==1) & (lap['prediction']==0)])) #fp

In [106]:
fp

34486.0

In [107]:
fn = float(len(lap[(lap['label']==0) & (lap['prediction']==1)])) #fn

In [108]:
fn

6165.0

In [109]:
! pip install plotly

In [110]:
import plotly.plotly as py
py.sign_in('grizzly', 'MHH5wgaO3T1lxbUFgeT5')
from plotly.graph_objs import *
import plotly.graph_objs as go
import pandas as pd
import requests
requests.packages.urllib3.disable_warnings()

### Sensitivity - True Positive Rate

### 99.6% of all cases who make fully paid are detected:

In [111]:
labels = ['TRUE-Fully Paid (TP)','FALSE-Charge Off (FN)']
values = [tp,fn]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')


### Specificity - True Negative Rate

### 3.09% of all cases who has been charged off are detected:

In [112]:
labels = ['TRUE-Charged Off (TN)','FALSE-Fully Paid (FP)']
values = [tn,fp]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

### Positive Predictive Value - Fully Paid

### the chance of 81.3% that a person with a positive test will be truly "fully paid".

In [113]:
labels = ['TRUE-Fully Paid (TP)','FALSE-Fully Paid (FP)']
values = [tp,fp]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

### Negative Predictive Value - Charged Off

### The chance of 66.7% that a person with a negative test will be truly "charged off".

In [114]:
labels = ['TRUE-Charged Off (TN)','FALSE-Charged Off (FN)']
values = [tn,fn]

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')